In [19]:
import numpy as np
import scipy as sp
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn import metrics, model_selection
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV

import nltk

<div align="right">
<a href="#placeholder" class="btn btn-default" data-toggle="collapse">Предварительные инструкции (clickable)</a>
</div>
<div id="placeholder" class="collapse">
<ol>
    <li>Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем большинстве этапов должны использоваться готовые протестированные инструменты. Все это приводит к тому, что baseline-модели - это дешевый способ сделать грубую оценку потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.</li>
    <li>Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе сделать предположения о том, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.</li>
    <li>Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.
</li>
    <li>Наконец, если после построение сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.
</li>
</ol>
</div>

In [2]:
df = pd.read_csv('data/products_sentiment_train.tsv', delimiter='\t', header=None, names=['sentences', 'p/n'])

In [3]:
df.info() # повезло, в данных нет пропусков.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
sentences    2000 non-null object
p/n          2000 non-null int64
dtypes: int64(1), object(1)
memory usage: 31.3+ KB


Для начала попробуем выделить признаки на основе счетчика слов и классифицировать их несколькими способами.
(LogisticRegression, SGDClassifier, LinearSVC)

In [74]:
def do_check(vectorizer, classifier): # just helper function
    pipe = Pipeline(steps = [
        ('vct', vectorizer),
        ('clf', classifier)
    ])
    
    # тут небольшой читинг, параметр фолдов для кросс валидации уже подобранн методом научного тыка.
    score = model_selection.cross_val_score(pipe, df.sentences, df['p/n'], cv=15, scoring='accuracy')
    return score.mean()

In [75]:
print("CV + LogisticRegression: {}".format(do_check(CountVectorizer(), LogisticRegression())))
print("CV + SGDClassifier:      {}".format(do_check(CountVectorizer(), SGDClassifier())))
print("CV + LinearSVC:          {}".format(do_check(CountVectorizer(), LinearSVC())))

CV + LogisticRegression: 0.7729754621742053
CV + SGDClassifier:      0.7459860642107302
CV + LinearSVC:          0.7484698290275666


Картина, которую мы наблюдаем с параметрами по умолчанию мягко говоря бездарная и что с этим делать пока не понятно, но попробуем то же, только на этот раз выделим признаки на основе частотности слов.

In [66]:
print("TV + LogisticRegression: {}".format(do_check(TfidfVectorizer(), LogisticRegression())))
print("TV + SGDClassifier:      {}".format(do_check(TfidfVectorizer(), SGDClassifier())))
print("TV + LinearSVC:          {}".format(do_check(TfidfVectorizer(), LinearSVC())))

TV + LogisticRegression: 0.7740194561482856
TV + SGDClassifier:      0.7539723914036561
TV + LinearSVC:          0.7699642026821917


Результат несколько улучшился, но он по прежнему бездарен. Нужно подбирать параметры. Но мы все еще можем испытать метод ближайших соседей и деревья.

In [67]:
from sklearn.neighbors import KNeighborsClassifier
print("Result with CV: {}".format(do_check(CountVectorizer(), KNeighborsClassifier())))
print("Result with TV: {}".format(do_check(TfidfVectorizer(), KNeighborsClassifier())))

Result with CV: 0.6589881533322225
Result with TV: 0.7249706128732052


In [69]:
from sklearn.tree import DecisionTreeClassifier
print("Result with CV: {}".format(do_check(CountVectorizer(), DecisionTreeClassifier())))
print("Result with TV: {}".format(do_check(TfidfVectorizer(), DecisionTreeClassifier())))

Result with CV: 0.6890227604288877
Result with TV: 0.6689460535100753


Это просто :facepalm:. В общем если мы не хотим сразу переключаться на lstm и подобные решения (вообще есть сомнения по поводу того, что они могут тут сильно улучшить результат, данных очень мало), единственный способ хоть как -то улучшиться на этом этапе, это все же подбор гиперпатамертов (В дальнейшем можно попробовать поработать с данными, но пока интересно понять, что можно получить "из коробки").

Сразу нужно уточнить, что на этапе baseline нам не интересно проводить глубокий анализ и потому мы просто возьмем лучшие результаты "из коробки" и для них уже будем пытаться проводить подбор параметров. Это частотность слов и методы опорных векторов, логистическая регрессия и стохастический градиентный спуск.

In [107]:
from time import time

def do_grid_search(pipeline, parameters):
    grid_search = GridSearchCV(pipeline, parameters, scoring='accuracy')
    t0 = time()
    grid_search.fit(df.sentences, df['p/n'])
    print("done in %0.3fs" % (time() - t0))
    
    print("Best score: %.4f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    return grid_search

In [151]:
tv, clf = TfidfVectorizer(), LogisticRegression()

pipe = Pipeline(steps = [
        ('vct', tv),
        ('clf', clf)
    ])

parameters = {
        'vct__max_df': (0.25, 0.5, 0.75, 1.0),
        'vct__max_features': (None, 5000, 10000, 50000),
        'vct__ngram_range': ((1, 3), (1, 2), (1, 5), (2, 3), (3, 5)),  # unigrams or bigrams
        'clf__C': (0.00001, 0.0001, 0.01, 1),
        'clf__penalty': ('l2', 'l1'),
}

gs = GridSearchCV(pipe, parameters, scoring='accuracy')

In [152]:
gs.fit(df.sentences, df['p/n'])

KeyboardInterrupt: 

In [150]:
best_params = gs.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_params[param_name]))

	clf__C: 1
	clf__penalty: 'l2'
	vct__max_df: 0.5
	vct__ngram_range: (1, 2)


In [77]:
vct = TfidfVectorizer(ngram_range=(1, 3))
clf = SGDClassifier(random_state=255)

pipe = Pipeline(steps = [
    ('vectorizer', vct),
    ('logistic', clf)
])

score = model_selection.cross_val_score(pipe, df.sentences, df['p/n'], cv=15, scoring='accuracy')

In [39]:
score.mean()

0.78951335824862923

In [ ]:
words = set(nltk.corpus.words.words())
stwords = nltk.corpus.stopwords.words('english')